# Note:
-(make sure both data1 and data2 have the same number of numerical and categorical columns)
- Run all the cells for data1
- Then, run all the cells for data2
- Then after you have the correlation matrices for both (data1 and data2) --> run the difference between matrices test
- This is the only way this file will work. If you dont follow the steps you will get a result of ZERO

In [1]:
import numpy as np
import pandas as pd
from numpy import linalg
from sklearn.model_selection import train_test_split

# Upload Dataset

In [3]:
# Dataset before augmentation
#data1 = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Ellee/Data/Stacked/stacked_orig.csv", header=None)
data1 = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Ellee/Data/Uniform/uniform_orig.csv", header=None)
#data1 = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Ellee/Data/Gaussian/gaussian_orig.csv", header=None)


# Dataset after augmentation
data2 = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Ellee/Data/Stacked/stacked_new.csv", header=None)
#data2 = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Ellee/Data/Uniform/uniform_new.csv", header=None)
#data2 = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Ellee/Data/Gaussian/gaussian_orig.csv", header=None)
#data2 = pd.read_csv("/Users/fabianafazio/Documents/GitHub/BP24/Ellee/Data/Gaussian/gaussian_new.csv", header=None)


# Check Shapes of Datasets (they must be the same)

In [7]:
# Print shapes for debugging
print(f"Shape of data1: {data1.shape}")
print(f"Shape of data2: {data2.shape}")


Shape of data1: (343, 25)
Shape of data2: (240, 25)


# Convert columns into Categorical (Only for Uniform Distribution)

In [9]:
# categorical columns have index: 2, 7, 10, 15, 24


# List of column indices that should be converted to categorical
categorical_columns = [2, 7, 10, 15, 24]

for i in categorical_columns:
    data1.iloc[:, i] = data1.iloc[:, i].astype('category') # Convert to categorical type

# Display the data types of the columns to verify the changes
print(data1.dtypes)


0      float64
1      float64
2     category
3      float64
4      float64
5      float64
6      float64
7     category
8      float64
9      float64
10    category
11     float64
12     float64
13     float64
14     float64
15    category
16     float64
17     float64
18     float64
19     float64
20     float64
21     float64
22     float64
23     float64
24    category
dtype: object


In [ ]:
# categorical columns have index: 2, 7, 10, 15, 24


# List of column indices that should be converted to categorical
categorical_columns = [2, 7, 10, 15, 24]

for i in categorical_columns:
    data2.iloc[:, i] = data2.iloc[:, i].astype('category') # Convert to categorical type

# Display the data types of the columns to verify the changes
print(data2.dtypes)


# Run this to make columns the same shape as Gaussian Distribution

In [ ]:
# subset 13 columns to keep the indexes above and match gaussian
# Indices of the columns to keep
primary_indices = [2, 7, 10, 15, 24]

# Calculate the additional columns needed
additional_indices = [i for i in range(data2.shape[1]) if i not in primary_indices][:8]

# Combine primary and additional indices
final_indices = primary_indices + additional_indices

# Subset the DataFrame
data2 = data2.iloc[:, final_indices]

# Display the new DataFrame
print(data2)


# Convert columns into Categorical (Only for Gaussian Distribution)

In [ ]:
# categorical columns have index: 2, 3, 7, 9, 12

# List of column indices that should be converted to categorical
categorical_columns = [2, 3, 7, 9, 12]

for i in categorical_columns:
    data1.iloc[:, i] = data1.iloc[:, i].astype('category') # Convert to categorical type

# Display the data types of the columns to verify the changes
print(data1.dtypes)


In [ ]:
# categorical columns have index: 2, 3, 7, 9, 12

# List of column indices that should be converted to categorical
categorical_columns = [2, 3, 7, 9, 12]

for i in categorical_columns:
    data2.iloc[:, i] = data2.iloc[:, i].astype('category') # Convert to categorical type

# Display the data types of the columns to verify the changes
print(data2.dtypes)


# Convert columns into Categorical (Only for Stacked Distribution)

In [ ]:
# if stacked is not the same shape as Gaussian, subset columns and match categorical columns to the distribution you are comparing
data2 = data2.iloc[:, -13:]
data2.shape

In [ ]:
# categorical columns have index: 16, 17, 18, 19, 20, 21, 22, 23, 24 (including label column)
for column in data1.columns[-9:]:
    data1[column] = data1[column].astype('category')

# Verify the changes
print(data1.dtypes)


In [13]:
# categorical columns have index: 16, 17, 18, 19, 20, 21, 22, 23, 24 (including label column)
for column in data2.columns[-9:]:
    data2[column] = data2[column].astype('category')

# Verify the changes
print(data2.dtypes)


0      float64
1      float64
2      float64
3      float64
4      float64
5      float64
6      float64
7      float64
8      float64
9      float64
10     float64
11     float64
12     float64
13     float64
14     float64
15     float64
16    category
17    category
18    category
19    category
20    category
21    category
22    category
23    category
24    category
dtype: object


# Data1: Synthetic Data

In [11]:
# data1
X = data1.iloc[:, :-1]
y = data1.iloc[:, -1]


# Split dataset into X_train and y_train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Splitting X_train into numerical subset 
numerical_df = X_train.select_dtypes(include = ['float', 'float64'])

# Splitting X_train into categorical subset 
categorical_df = X_train.select_dtypes(exclude=['float', 'float64'])


##################### Correlation between columns (numerical) Code ############################
# Takes the X_train data to find correlation between NUMERICAL features
def num_corr(X_train_numerical):
    matrix = X_train_numerical.corr(method='pearson')
    print("---------------------------Correlation Matrix------------------------- \n", matrix)
     
#Calls the function so the matrix prints out    
correlation_matrix = num_corr(numerical_df)



#################### save Data1 matrix as a dataframe #################################
#data1
correlation_matrix = numerical_df.corr()
correlation_df1 = pd.DataFrame(correlation_matrix)
print(correlation_df1)

correlation_df1.shape

---------------------------Correlation Matrix------------------------- 
           0         1         3         4         5         6         8   \
0   1.000000  0.153738  0.084442  0.065896  0.182015  0.132600 -0.007064   
1   0.153738  1.000000  0.146102  0.093639  0.203931  0.174417  0.084242   
3   0.084442  0.146102  1.000000  0.086641  0.160223  0.137962  0.064725   
4   0.065896  0.093639  0.086641  1.000000  0.190358  0.044607  0.025051   
5   0.182015  0.203931  0.160223  0.190358  1.000000  0.114466  0.156622   
6   0.132600  0.174417  0.137962  0.044607  0.114466  1.000000 -0.003306   
8  -0.007064  0.084242  0.064725  0.025051  0.156622 -0.003306  1.000000   
9   0.086244  0.105229  0.158098  0.098994  0.092857  0.128217  0.059255   
11  0.065992  0.114128  0.034905  0.178465  0.110940  0.000335  0.127832   
12  0.084564  0.188954  0.088043  0.171399  0.146997  0.189323  0.156012   
13  0.097929  0.086318  0.057185  0.099178  0.175591  0.178317  0.080372   
14  0.129875  0

(20, 20)

# Data1 + Data2: Combined_data

In [17]:
########################## Concat Data1 +  Data2 ##################################
# Concat (adding rows)
combined_df = np.vstack((data1, data2))

# change numpy array into a dataframe
combined_data = pd.DataFrame(combined_df, columns=data1.columns)

# Convert the last number of columns to categorical (to make sure they have the same)
for column in combined_data.columns[-9:]:
    combined_data[column] = combined_data[column].astype('category')

# Verify the changes
print(combined_data.dtypes)


##################### Split Combined_data ################################

# Split dataset into X and y (data1)
X = combined_data.iloc[:, :-1]  # All columns except the last one
y = combined_data.iloc[:, -1]   # Only the last column


# Split dataset into X_train and y_train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Splitting X_train into numerical subset 
numerical_df = X_train.select_dtypes(include = ['float', 'float64'])

# Splitting X_train into categorical subset 
categorical_df = X_train.select_dtypes(exclude=['float', 'float64'])


##################### Correlation between columns (numerical) Code ############################
# Takes the X_train data to find correlation between NUMERICAL features
def num_corr(X_train_numerical):
    matrix = X_train_numerical.corr(method='pearson')
    print("---------------------------Correlation Matrix------------------------- \n", matrix)
     
#Calls the function so the matrix prints out    
correlation_matrix = num_corr(numerical_df)


############################## save combined_data matrix as a dataframe ############################
#combined_data
correlation_matrix = numerical_df.corr()
correlation_df_combined = pd.DataFrame(correlation_matrix)
print(correlation_df_combined)

correlation_df_combined.shape


0      float64
1      float64
2      float64
3      float64
4      float64
5      float64
6      float64
7      float64
8      float64
9      float64
10     float64
11     float64
12     float64
13     float64
14     float64
15     float64
16    category
17    category
18    category
19    category
20    category
21    category
22    category
23    category
24    category
dtype: object
---------------------------Correlation Matrix------------------------- 
           0         1         2         3         4         5         6   \
0   1.000000  0.125908  0.257749  0.192079  0.197068  0.209122  0.336646   
1   0.125908  1.000000  0.198190  0.256835  0.246989  0.218769  0.225915   
2   0.257749  0.198190  1.000000  0.210905  0.206674  0.225367  0.241126   
3   0.192079  0.256835  0.210905  1.000000  0.278856  0.192772  0.227992   
4   0.197068  0.246989  0.206674  0.278856  1.000000  0.352564  0.392056   
5   0.209122  0.218769  0.225367  0.192772  0.352564  1.000000  0.272655   
6   0.

(16, 16)

# Differences in Matrices (Frobenius Norm)

In [ ]:
# Convert the dataframes to numpy arrays
matrix1 = correlation_df1.to_numpy()
matrix2 = correlation_df_combined.to_numpy()

# Compute the Frobenius norm of the difference between the matrices. 
frobenius_abs = np.linalg.norm(matrix1 - matrix2, ord='fro')   # Absolute error with Frobenius norm

frobenius_rel = frobenius_abs/np.linalg.norm(matrix1, ord='fro')    # Relative error with Frobenius norm

print(f"Frobenius norm (absolute error) : {frobenius_abs: .3f}")
print(f"Frobenius norm (relative error) : {frobenius_rel: .3f}")


# Differences in Arrays (L2 Norm)

In [ ]:
#array1 ...
#array2 ...

# Compute the L2 norm (Euclidean norm) of the difference between the arrays
L2_norm = np.linalg.norm(array1 - array2, ord=2)

print(f"L2 norm difference: {L2_norm: .3f}")
